*TCP-Client*

In [9]:
import socket

target_host = "www.google.com"
target_port = 80

# create a socket object
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# connect the client
client.connect((target_host, target_port))

# send some data
request = b"GET / HTTP/1.1\r\nHost: google.com\r\n\r\n"
client.send(request)

# receive some data
response = client.recv(4096)
print(response.decode())

HTTP/1.1 301 Moved Permanently
Location: http://www.google.com/
Content-Type: text/html; charset=UTF-8
Content-Security-Policy-Report-Only: object-src 'none';base-uri 'self';script-src 'nonce-hH8xol76XlkhNNOiHscHIg' 'strict-dynamic' 'report-sample' 'unsafe-eval' 'unsafe-inline' https: http:;report-uri https://csp.withgoogle.com/csp/gws/other-hp
Date: Sat, 03 Jun 2023 17:21:33 GMT
Expires: Mon, 03 Jul 2023 17:21:33 GMT
Cache-Control: public, max-age=2592000
Server: gws
Content-Length: 219
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN

<HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">
<TITLE>301 Moved</TITLE></HEAD><BODY>
<H1>301 Moved</H1>
The document has moved
<A HREF="http://www.google.com/">here</A>.
</BODY></HTML>



*UDP-Client*

In [6]:
import socket

target_host = "127.0.0.1"
target_port = 9000

# create a socket object
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    # try to connect to the target host and port
    client.connect((target_host, target_port))
    print(f"Port {target_port} is open and ready to receive data.")
except ConnectionRefusedError:
    print(f"Port {target_port} is not open or not ready to receive data.")
finally:
    # send some data
    client.sendto(b"AAABBBCCC", (target_host, target_port))

    # receive some data
    data, addr = client.recvfrom(4096)
    
   
    print("Received data:", data)
    
    client.close()


Port 9000 is open and ready to receive data.
Received data: b'\xff\x00\x00\x00\x00\x00\x00\x00\x01\x7f'


*TCP-Server*

In [4]:
import socket
import threading

bind_ip = "0.0.0.0"
bind_port = 9012

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((bind_ip, bind_port))
server.listen(5)
print(f"[*] Listening on {bind_ip}:{bind_port}")

# Esta es nuestra función para manejar al cliente
def handle_client(client_socket):
    # Imprimimos lo que el cliente envía
    request = client_socket.recv(1024)
    print(f"[*] Received: {request.decode()}")
    # Enviamos de vuelta un paquete
    client_socket.send(b"ACK!")
    client_socket.close()

while True:
    client, addr = server.accept()
    print(f"[*] Accepted connection from: {addr[0]}:{addr[1]}")
    # Creamos un hilo para manejar los datos entrantes del cliente
    client_handler = threading.Thread(target=handle_client, args=(client,))
    client_handler.start()


[*] Listening on 0.0.0.0:9012


In [ ]:
import socket

target_ip = "0.0.0.0"  # IP del servidor
target_port = 9012      # Puerto del servidor

# Creamos un objeto de socket para el cliente
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Conectamos el cliente al servidor
client.connect((target_ip, target_port))

# Enviamos un mensaje al servidor
message = "ABCDEF"
client.send(message.encode())

# Recibimos la respuesta del servidor
response = client.recv(1024)
print(f"[*] Received: {response.decode()}")

# Cerramos la conexión del cliente
client.close()


ConnectionRefusedError: [Errno 111] Connection refused